In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-ai-hackathon-2025/sample_submission.csv
/kaggle/input/ml-ai-hackathon-2025/train_labels.csv
/kaggle/input/ml-ai-hackathon-2025/train.csv
/kaggle/input/ml-ai-hackathon-2025/test.csv


In [3]:
#to avoid warnings
import warnings
warnings.filterwarnings("ignore")

**LOADING DATASETS AND PREPROCESSING**

In [4]:
#load training data
train_df = pd.read_csv('/kaggle/input/ml-ai-hackathon-2025/train.csv')

In [5]:
train_df.head()

,Id,gene_1,gene_3,gene_5,gene_7,gene_8,gene_9,gene_10,gene_11,gene_13,...,gene_20634,gene_20635,gene_20636,gene_20637,gene_20638,gene_20639,gene_20640,gene_20641,gene_20642,Class
0,sample_664,0.160738,-0.327348,-0.144638,0.196493,-1.105093,0.309926,-0.177461,-1.124182,-0.459826,...,-1.611378,-1.108411,-0.670719,-1.739299,0.476467,1.136071,-0.576601,-1.275518,-0.508678,1.0
1,sample_215,-0.771173,0.885819,-0.234209,0.273139,0.132208,-0.249541,0.005817,-0.631647,NaN,...,0.247812,0.144035,0.148776,-1.373208,0.099245,0.391993,0.573363,0.322198,6.022439,0.0
2,sample_343,-0.169258,1.908618,0.165008,-0.562826,0.199720,0.128036,2.348450,2.425346,-0.933545,...,1.133065,0.965014,1.873753,-0.005167,-0.223091,0.782868,-0.562787,-0.471593,-0.763284,4.0
3,sample_707,-0.947912,0.111177,-0.153179,0.837412,0.185467,-0.066223,-0.267734,0.674365,-0.076086,...,0.022339,0.326506,-0.333964,0.228595,-0.245309,0.478564,0.273364,1.756369,-0.266200,1.0
4,sample_621,-0.335741,0.515251,0.325440,-0.842387,-0.500415,0.484240,-0.438587,-0.874562,NaN,...,-1.516812,-1.430622,-0.664933,-0.753410,NaN,0.375521,-0.536705,-0.523850,0.222560,4.0


In [6]:
#shape of the training data and number of columns
print('training data shape',train_df.shape)
print(f"Columns: Id + {train_df.shape[1]-2} gene features + Class")

training data shape (400, 14574)
Columns: Id + 14572 gene features + Class


In [7]:
# Load labeled subset data
train_labels_df=pd.read_csv('/kaggle/input/ml-ai-hackathon-2025/train_labels.csv')

In [8]:
train_labels_df.head()

,Id,Class
0,sample_664,1
1,sample_215,0
2,sample_343,4
3,sample_707,1
4,sample_621,4


In [9]:
#shape of the training subset data
train_labels_df.shape

(150, 2)

In [10]:
train_labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      150 non-null    object
 1   Class   150 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [11]:
# Check labeled vs unlabeled samples in train.csv
labeled_count = train_df['Class'].notna().sum()
unlabeled_count = train_df['Class'].isna().sum()
print(f"\nIn train.csv:")
print(f"Labeled samples: {labeled_count}")
print(f"Unlabeled samples: {unlabeled_count}")


In train.csv:
Labeled samples: 150
Unlabeled samples: 250


In [12]:
# Class distribution in labeled data

print(f"\nClass distribution in labeled data:")
print(train_labels_df['Class'].value_counts().sort_index())


Class distribution in labeled data:
Class
0    26
1    26
2    56
3    15
4    27
Name: count, dtype: int64


In [13]:
#prepare labeled training data
labeled_train_df=train_df.merge(train_labels_df, on='Id')

In [14]:
labeled_train_df.head()

,Id,gene_1,gene_3,gene_5,gene_7,gene_8,gene_9,gene_10,gene_11,gene_13,...,gene_20635,gene_20636,gene_20637,gene_20638,gene_20639,gene_20640,gene_20641,gene_20642,Class_x,Class_y
0,sample_664,0.160738,-0.327348,-0.144638,0.196493,-1.105093,0.309926,-0.177461,-1.124182,-0.459826,...,-1.108411,-0.670719,-1.739299,0.476467,1.136071,-0.576601,-1.275518,-0.508678,1.0,1
1,sample_215,-0.771173,0.885819,-0.234209,0.273139,0.132208,-0.249541,0.005817,-0.631647,NaN,...,0.144035,0.148776,-1.373208,0.099245,0.391993,0.573363,0.322198,6.022439,0.0,0
2,sample_343,-0.169258,1.908618,0.165008,-0.562826,0.199720,0.128036,2.348450,2.425346,-0.933545,...,0.965014,1.873753,-0.005167,-0.223091,0.782868,-0.562787,-0.471593,-0.763284,4.0,4
3,sample_707,-0.947912,0.111177,-0.153179,0.837412,0.185467,-0.066223,-0.267734,0.674365,-0.076086,...,0.326506,-0.333964,0.228595,-0.245309,0.478564,0.273364,1.756369,-0.266200,1.0,1
4,sample_621,-0.335741,0.515251,0.325440,-0.842387,-0.500415,0.484240,-0.438587,-0.874562,NaN,...,-1.430622,-0.664933,-0.753410,NaN,0.375521,-0.536705,-0.523850,0.222560,4.0,4


In [15]:
#shape of the labeled training data
labeled_train_df.shape

(150, 14575)

In [16]:
#drop extra class column
labeled_train_df.drop(columns={'Class_x','Id'},inplace=True)

In [17]:
labeled_train_df.head()

,gene_1,gene_3,gene_5,gene_7,gene_8,gene_9,gene_10,gene_11,gene_13,gene_14,...,gene_20634,gene_20635,gene_20636,gene_20637,gene_20638,gene_20639,gene_20640,gene_20641,gene_20642,Class_y
0,0.160738,-0.327348,-0.144638,0.196493,-1.105093,0.309926,-0.177461,-1.124182,-0.459826,-1.111016,...,-1.611378,-1.108411,-0.670719,-1.739299,0.476467,1.136071,-0.576601,-1.275518,-0.508678,1
1,-0.771173,0.885819,-0.234209,0.273139,0.132208,-0.249541,0.005817,-0.631647,NaN,0.540441,...,0.247812,0.144035,0.148776,-1.373208,0.099245,0.391993,0.573363,0.322198,6.022439,0
2,-0.169258,1.908618,0.165008,-0.562826,0.199720,0.128036,2.348450,2.425346,-0.933545,-0.968164,...,1.133065,0.965014,1.873753,-0.005167,-0.223091,0.782868,-0.562787,-0.471593,-0.763284,4
3,-0.947912,0.111177,-0.153179,0.837412,0.185467,-0.066223,-0.267734,0.674365,-0.076086,-0.321164,...,0.022339,0.326506,-0.333964,0.228595,-0.245309,0.478564,0.273364,1.756369,-0.266200,1
4,-0.335741,0.515251,0.325440,-0.842387,-0.500415,0.484240,-0.438587,-0.874562,NaN,NaN,...,-1.516812,-1.430622,-0.664933,-0.753410,NaN,0.375521,-0.536705,-0.523850,0.222560,4


In [18]:
# rename class label from Class_y to Class
labeled_train_df.rename(columns={'Class_y':'Class'},inplace=True)

In [19]:
#checking null values in the labeled training data
labeled_train_df.isnull().sum()

gene_1         6
gene_3         5
gene_5        20
gene_7        13
gene_8        13
              ..
gene_20639    12
gene_20640    17
gene_20641    14
gene_20642    11
Class          0
Length: 14573, dtype: int64

In [20]:
# Extract gene features (all columns except Id and Class)

gene_cols = [col for col in train_df.columns if col.startswith('gene_')]
print(f"\nNumber of gene features: {len(gene_cols)}")


Number of gene features: 14572


In [21]:
# Load test data (401 samples for prediction)

test_df = pd.read_csv('/kaggle/input/ml-ai-hackathon-2025/test.csv') 
print(f"Test data shape: {test_df.shape}")
print(f"Columns: Id + {test_df.shape[1]-1} gene features")

Test data shape: (401, 14573)
Columns: Id + 14572 gene features


In [22]:
test_df.head()

,Id,gene_1,gene_3,gene_5,gene_7,gene_8,gene_9,gene_10,gene_11,gene_13,...,gene_20633,gene_20634,gene_20635,gene_20636,gene_20637,gene_20638,gene_20639,gene_20640,gene_20641,gene_20642
0,sample_313,0.554589,-1.758558,-0.822430,-0.453578,0.355311,2.174823,-0.746382,2.796776,1.356738,...,0.814713,-0.917544,-1.174990,0.230343,1.160923,0.083997,1.631368,-0.530967,0.154798,0.141265
1,sample_427,1.159732,0.584532,-0.290216,-0.308697,0.858271,0.427662,0.880553,1.083369,-0.617963,...,0.544172,-1.447488,-0.624379,-1.793731,0.098738,0.677765,-0.979769,-0.840423,-0.186195,-0.402395
2,sample_573,-0.241096,-0.523115,-0.464614,-0.413843,-0.024800,-0.020333,-0.236695,-0.448820,NaN,...,1.959343,1.188033,-0.378787,0.890212,-3.607169,-0.071064,-2.401333,0.593609,0.337543,0.285292
3,sample_214,1.277153,0.625642,-0.280942,0.290133,-0.296918,0.784187,-0.300130,-1.063136,0.190820,...,-0.049144,-0.832593,-1.436841,0.404060,0.177354,NaN,0.024182,NaN,-1.360115,0.907782
4,sample_793,0.691170,-0.243284,-0.357550,0.348433,-0.653095,0.655224,0.236587,NaN,0.608754,...,0.935813,0.694044,0.590808,0.153625,-0.228556,-1.723097,0.933723,-0.923859,1.367854,-0.302595


In [23]:
# Prepare test data
X_test = test_df[gene_cols]
test_ids = test_df['Id']

print(f"Test features shape: {X_test.shape}")
print(f"Test IDs shape: {test_ids.shape}")

Test features shape: (401, 14572)
Test IDs shape: (401,)


In [24]:
X_test.isnull().sum()

gene_1        38
gene_3        25
gene_5        32
gene_7        43
gene_8        36
              ..
gene_20638    45
gene_20639    28
gene_20640    34
gene_20641    42
gene_20642    26
Length: 14572, dtype: int64

# Feature Engineering

In [25]:
#Features and target
X_train = labeled_train_df[gene_cols]
y_train = labeled_train_df['Class']

In [26]:
X_train.shape

(150, 14572)

In [27]:
X_train.head(2)

,gene_1,gene_3,gene_5,gene_7,gene_8,gene_9,gene_10,gene_11,gene_13,gene_14,...,gene_20633,gene_20634,gene_20635,gene_20636,gene_20637,gene_20638,gene_20639,gene_20640,gene_20641,gene_20642
0,0.160738,-0.327348,-0.144638,0.196493,-1.105093,0.309926,-0.177461,-1.124182,-0.459826,-1.111016,...,-0.567048,-1.611378,-1.108411,-0.670719,-1.739299,0.476467,1.136071,-0.576601,-1.275518,-0.508678
1,-0.771173,0.885819,-0.234209,0.273139,0.132208,-0.249541,0.005817,-0.631647,NaN,0.540441,...,0.506589,0.247812,0.144035,0.148776,-1.373208,0.099245,0.391993,0.573363,0.322198,6.022439


We using KNN imputer for imputing NULL values in the dataset since KNN can captures non-linear patterns in the data and preseves Data relationships whereas simple imuter(mean/median) can't perform that much.

In [28]:
#imputing null values using KNN imputer
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=3)
X_imputed = knn_imputer.fit_transform(X_train)
X_test_imputed = knn_imputer.fit_transform(X_test)

In [29]:
X_imputed

array([[ 0.16073775, -0.32734836, -0.14463841, ..., -0.57660081,
        -1.27551822, -0.50867754],
       [-0.77117295,  0.88581926, -0.23420886, ...,  0.5733627 ,
         0.32219824,  6.02243904],
       [-0.16925751,  1.90861811,  0.16500842, ..., -0.56278672,
        -0.4715927 , -0.76328382],
       ...,
       [-0.00962615, -0.03767667,  0.51786593, ...,  0.67718853,
        -0.12292465, -0.49285547],
       [ 0.496826  , -0.37091238,  0.81045268, ...,  0.115815  ,
         0.49673597,  0.38730922],
       [ 1.11428755,  0.52190298, -0.09111339, ...,  0.36529583,
         1.42221772,  0.62856574]])

In [30]:
X_imputed.shape

(150, 14572)

In [31]:
y_train.shape

(150,)

In [32]:
X_test_imputed

array([[ 0.55458853, -1.7585578 , -0.82243042, ..., -0.53096661,
         0.15479808,  0.14126478],
       [ 1.15973187,  0.58453231, -0.29021639, ..., -0.84042296,
        -0.1861947 , -0.40239478],
       [-0.24109622, -0.52311532, -0.46461352, ...,  0.59360867,
         0.33754271,  0.28529154],
       ...,
       [ 0.14221458, -1.64264903,  0.92421057, ...,  0.92217548,
         2.32392758, -0.57588142],
       [ 0.18463672, -0.3105085 , -0.59656022, ..., -1.44345356,
         0.1419881 , -1.2328941 ],
       [ 0.75758631,  1.06768652, -0.18705949, ..., -0.94687629,
        -0.94672781,  0.60269472]])

Using StandardScaler, to scale the whole data because StandardScaler standardizes features by removing the mean and scaling to unit variance.

In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_imputed)
X_test_scaled = scaler.fit_transform(X_test_imputed)

In [34]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(150, 14572)
(401, 14572)


Using PCA to reduce high dimensional to low dimensional data to reduce complexity of model training.

In [35]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 100)
X_pca = pca.fit_transform(X_train_scaled, y_train)
X_test_pca = pca.transform(X_test_scaled)

In [36]:
print(X_pca.shape)
print(X_test_pca.shape)

(150, 100)
(401, 100)


SelectKBest work similar like PCA does dimensionality reduction but SelectKBest reduce dimensions by choosing the most relevant features for the target whereas PCA reduce dimensions by compressing information (variance).

In [37]:
from sklearn.feature_selection import SelectKBest, f_classif

k=500
selector = SelectKBest(score_func = f_classif, k=k)
X_selected = selector.fit_transform(X_train_scaled,y_train)
X_test_selected = selector.transform(X_test_scaled)

In [38]:
print(X_selected.shape)
print(X_test_selected.shape)

(150, 500)
(401, 500)


In [39]:
# TODO : Train your Model for making predictions on test data

# Model Training

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPClassifier

In [41]:
#MODELS

model_rf= RandomForestClassifier(n_estimators= 100, random_state= 42, class_weight = 'balanced')
model_svm = SVC(kernel ='rbf',class_weight='balanced',probability=True)
model_xgb = XGBClassifier(use_label_encoder = False, eval_metric = 'mlogloss', random_state =42)
model_lr = LogisticRegression(penalty='l2', C=1.0, class_weight='balanced', random_state=42)
model_NN= MLPClassifier(hidden_layer_sizes = (256,128), activation= 'relu',max_iter=300, random_state=42)

In [42]:
# raw(SCALED DATA) model training

model_rf.fit(X_train_scaled, y_train)
model_svm.fit(X_train_scaled,y_train)
model_xgb.fit(X_train_scaled,y_train)
model_lr.fit(X_train_scaled,y_train)
model_NN.fit(X_train_scaled, y_train)

y_pred_cv_rf_ = cross_val_predict(model_rf, X_train_scaled, y_train, cv=5)
y_pred_cv_svm_ = cross_val_predict(model_svm, X_train_scaled, y_train, cv=5)
y_pred_cv_xgb_ = cross_val_predict(model_xgb, X_train_scaled, y_train, cv=5)
y_pred_cv_lr_ = cross_val_predict(model_lr, X_train_scaled, y_train, cv=5)
y_pred_cv_NN_ = cross_val_predict(model_NN, X_train_scaled, y_train, cv=5)

In [43]:
# model training after PCA
model_rf.fit(X_pca, y_train)
model_svm.fit(X_pca,y_train)
model_xgb.fit(X_pca,y_train)
model_lr.fit(X_pca,y_train)
model_NN.fit(X_pca, y_train)

# Generate cross-validated predictions
y_pred_cv_rf = cross_val_predict(model_rf, X_pca, y_train, cv=5)
y_pred_cv_svm = cross_val_predict(model_svm, X_pca, y_train, cv=5)
y_pred_cv_xgb = cross_val_predict(model_xgb, X_pca, y_train, cv=5)
y_pred_cv_lr = cross_val_predict(model_lr, X_pca, y_train, cv=5)
y_pred_cv_NN = cross_val_predict(model_NN, X_pca, y_train, cv=5)

In [44]:
# model training after SelectKBest

model_rf.fit(X_selected, y_train)
model_svm.fit(X_selected,y_train)
model_xgb.fit(X_selected,y_train)
model_lr.fit(X_selected,y_train)
model_NN.fit(X_selected, y_train)

y_pred_cv_rf__ = cross_val_predict(model_rf, X_selected, y_train, cv=5)
y_pred_cv_svm__ = cross_val_predict(model_svm, X_selected, y_train, cv=5)
y_pred_cv_xgb__ = cross_val_predict(model_xgb, X_selected, y_train, cv=5)
y_pred_cv_lr__ = cross_val_predict(model_lr, X_selected, y_train, cv=5)
y_pred_cv_NN__ = cross_val_predict(model_NN, X_selected, y_train, cv=5)

# Classification Report

In [45]:
#Classification Report for RAW(SCALED) Data 
print("Classification Report of Random Forest : \n",classification_report(y_train, y_pred_cv_rf_))
print("Classification Report of SVM : \n" ,classification_report(y_train, y_pred_cv_svm_))
print("Classification Report of XGBoost : \n",classification_report(y_train, y_pred_cv_xgb_))
print("Classification Report of Logistic Regression : \n",classification_report(y_train, y_pred_cv_lr_))
print("Classification Report of NN : \n",classification_report(y_train, y_pred_cv_NN_))

Classification Report of Random Forest : 
               precision    recall  f1-score   support

           0       1.00      0.96      0.98        26
           1       1.00      0.92      0.96        26
           2       0.93      1.00      0.97        56
           3       1.00      1.00      1.00        15
           4       1.00      0.96      0.98        27

    accuracy                           0.97       150
   macro avg       0.99      0.97      0.98       150
weighted avg       0.98      0.97      0.97       150

Classification Report of SVM : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       0.93      0.96      0.94        26
           2       0.98      1.00      0.99        56
           3       1.00      0.93      0.97        15
           4       1.00      0.96      0.98        27

    accuracy                           0.98       150
   macro avg       0.98      0.97      0.98       150
w

In [55]:
# Classification Report for PCA trained Data 
print("Classification Report of Random Forest : \n",classification_report(y_train, y_pred_cv_rf))
print("Classification Report of SVM : \n" ,classification_report(y_train, y_pred_cv_svm))
print("Classification Report of XGBoost : \n",classification_report(y_train, y_pred_cv_xgb))
print("Classification Report of Logistic Regression : \n",classification_report(y_train, y_pred_cv_lr))
print("Classification Report of NN : \n",classification_report(y_train, y_pred_cv_NN))

Classification Report of Random Forest : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       0.93      0.96      0.94        26
           2       1.00      1.00      1.00        56
           3       1.00      0.93      0.97        15
           4       0.96      0.96      0.96        27

    accuracy                           0.98       150
   macro avg       0.98      0.97      0.97       150
weighted avg       0.98      0.98      0.98       150

Classification Report of SVM : 
               precision    recall  f1-score   support

           0       1.00      0.73      0.84        26
           1       0.90      0.69      0.78        26
           2       0.70      1.00      0.82        56
           3       1.00      0.53      0.70        15
           4       1.00      0.85      0.92        27

    accuracy                           0.83       150
   macro avg       0.92      0.76      0.81       150
w

In [50]:
# Classification Report for SelectKBest trained Data
print("Classification Report of Random Forest : \n",classification_report(y_train, y_pred_cv_rf__))
print("Classification Report of SVM : \n" ,classification_report(y_train, y_pred_cv_svm__))
print("Classification Report of XGBoost : \n",classification_report(y_train, y_pred_cv_xgb__))
print("Classification Report of Logistic Regression : \n",classification_report(y_train, y_pred_cv_lr__))
print("Classification Report of NN : \n",classification_report(y_train, y_pred_cv_NN__))

Classification Report of Random Forest : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       1.00      0.92      0.96        26
           2       0.95      1.00      0.97        56
           3       1.00      1.00      1.00        15
           4       1.00      0.96      0.98        27

    accuracy                           0.98       150
   macro avg       0.99      0.98      0.98       150
weighted avg       0.98      0.98      0.98       150

Classification Report of SVM : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       1.00      0.96      0.98        26
           2       0.97      1.00      0.98        56
           3       1.00      1.00      1.00        15
           4       1.00      0.96      0.98        27

    accuracy                           0.99       150
   macro avg       0.99      0.98      0.99       150
w

 # Predictions on Trained Model 

In [46]:
#Generate predictions using RAW trained model

pred_rf_ = model_rf.predict(X_test_scaled)
pred_svc_ = model_svm.predict(X_test_scaled)
pred_xgb_ = model_xgb.predict(X_test_scaled)
pred_lr_ = model_lr.predict(X_test_scaled)
pred_NN_ = model_NN.predict(X_test_scaled)



In [56]:
#Generate predictions using PCA trained model

pred_rf = model_rf.predict(X_test_pca)
pred_svc = model_svm.predict(X_test_pca)
pred_xgb = model_xgb.predict(X_test_pca)
pred_lr = model_lr.predict(X_test_pca)
pred_NN = model_NN.predict(X_test_pca)

In [51]:
#Generate predictions using SelectKBest trained model

pred_rf__ = model_rf.predict(X_test_selected)
pred_svc__ = model_svm.predict(X_test_selected)
pred_xgb__ = model_xgb.predict(X_test_selected)
pred_lr__ = model_lr.predict(X_test_selected)
pred_NN__ = model_NN.predict(X_test_selected)

In [57]:
# For now, creating dummy predictions (replace this with your model predictions on X_test)
predictions = np.random.choice([0, 1, 2, 3, 4], size=len(X_test_imputed))

In [58]:
predictions

array([0, 4, 1, 4, 3, 0, 2, 1, 3, 4, 2, 2, 0, 2, 2, 4, 3, 4, 2, 3, 4, 2,
       4, 4, 0, 2, 1, 3, 2, 4, 1, 0, 4, 1, 0, 0, 2, 1, 4, 1, 0, 1, 1, 1,
       1, 3, 3, 0, 1, 2, 1, 2, 4, 0, 0, 0, 2, 1, 0, 2, 4, 3, 0, 4, 4, 1,
       3, 3, 0, 0, 1, 1, 2, 4, 1, 2, 1, 3, 0, 0, 4, 0, 3, 3, 4, 2, 0, 1,
       3, 3, 0, 1, 1, 4, 4, 1, 0, 1, 3, 3, 4, 0, 3, 4, 4, 2, 3, 1, 2, 0,
       2, 0, 4, 4, 4, 2, 4, 0, 1, 0, 1, 4, 3, 3, 1, 1, 4, 2, 0, 0, 0, 2,
       2, 4, 2, 4, 4, 0, 2, 4, 2, 1, 2, 4, 1, 3, 2, 0, 1, 1, 4, 1, 1, 2,
       2, 2, 4, 4, 4, 1, 0, 4, 1, 4, 3, 0, 0, 1, 1, 2, 3, 4, 2, 3, 3, 3,
       2, 2, 2, 0, 0, 4, 3, 0, 2, 4, 1, 4, 0, 3, 4, 1, 2, 0, 3, 1, 0, 0,
       3, 4, 2, 3, 1, 1, 3, 1, 2, 2, 3, 3, 1, 1, 2, 4, 1, 3, 4, 4, 2, 2,
       0, 0, 1, 2, 1, 0, 4, 3, 4, 0, 2, 2, 0, 3, 3, 3, 2, 2, 0, 2, 0, 0,
       1, 4, 2, 3, 0, 0, 4, 2, 1, 1, 1, 0, 4, 1, 2, 4, 2, 2, 4, 4, 2, 2,
       4, 0, 1, 1, 3, 0, 4, 1, 2, 1, 4, 0, 2, 1, 3, 3, 2, 2, 2, 4, 2, 0,
       4, 0, 3, 2, 3, 2, 2, 1, 3, 2, 0, 4, 2, 4, 3,

# In this Project, the Goal was to build a robust classification model for high-dimensional biological data with missing values. After extensive preprocessing , feature engineering and model experimentation , the final model chosen was a Random Forest Classifier trained on features selected using SelectKBest.

# Several machine learning models were evaluated including Logistic Regression, SVM , XGBoost , Random Forest and Neural Network.
# Performance was measured using cross-validated F1-score, precision and recall. While many models schived high scores, Random Forest consisitenly outperformed others , especially when combined with SelectKBest.

# final model Random Forsr Classifier trained on SelectKBest reduced features, chosen for its excellent predictive perfomance , robustness to high-dimensionality and ease of interpretability. PCA was not used in the final model, as they either reduced performance or were uncessary for tree- bases classifiers.

In [61]:
submission = pd.DataFrame({
    'Id': test_ids,
    'Class': pred_rf__
})

# Save submission file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file created!")
print(f"Submission shape: {submission.shape}")


Submission file created!
Submission shape: (401, 2)
